# Tutorial for MDUS
MDUSのtutorialコード

## STEP 0：データのダウンロード
MDUSでは以下のデータに対応しています
* MESSENGER MAG Time-Averaged Calibrated MSO Coordinates Science Data Collection
* MESSENGER EPPS EPS Calibrate Scan Rate Data Collection

事前に各自でデータをダウンロードしてくる必要があります

ダウンロードしたデータは適当なディレクトリに配置する必要があります  
本チュートリアルでは以下のようなディレクトリ構造を想定します        
ここではダウンロードしてきたデータを`./Data/ASCII/MAG`などに配置します      

```shell
.
├── tutorial.ipynb
├── datapath.json
└── Data
    ├── ASCII
    │   ├── MAG
    │   └── FIPS_CDR_SCAN
    └── Binary
        ├── MAG
        └── FIPS_CDR_SCAN

```

In [2]:
# MDUSのインポート
import MDUS

## STEP1：初回起動
MDUSを初回起動した際には以下のようなメッセージが表示されます
```shell
Some spkernel files are missing
You cannnot use some following functions:
Missing files:
    msgr_040803_150430_150430_od431sc_2.bsp
    msgr_dyn_v600.tf
    naif0012.tls
    pck00010_msgr_v23.tpc
datapath.json file is missing
create a new datapath.json file
```
MDUSを利用するためには以下の２つを行わなければいけません
1. SPICEカーネルの準備
2. datapath.jsonの準備


### SPICEカーネルの準備
MESSENGER衛星の軌道データを取得するために，SPICEカーネルが必要になります    
SPICEカーネルのダウンロードには`MDUS.DownloadSPK()`を利用します

In [ ]:
# SPICEカーネルのダウンロード
# naif0012.tlsがない場合
MDUS.DownloadSPK()

Already exists: msgr_040803_150430_150430_od431sc_2.bsp
Already exists: msgr_dyn_v600.tf
Downloading: naif0012.tls
Successfully downloaded: naif0012.tls
Already exists: pck00010_msgr_v23.tpc


`msgr_040803_150430_150430_od431sc_2.bsp`，`msgr_dyn_v600.tf`，`naif0012.tls`，`pck00010_msgr_v23.tpc`の4ファイルがダンロードできれば成功です

以降，MDUS利用の際にSPICEカーネルの利用が可能になります     
本操作は，MDUSを初めて利用する際に一度だけ行えば大丈夫です（別のディレクトリ
で作業する際には実行する必要なし）

### datapath.jsonの準備
MDUSでMESSENGERデータを利用可能にするために，`datapath.json`の設定が必要になります  
初回起動時には以下のような内容のファイルが生成されます
```shell
{
    "MAG": {
        "pfile_path": "/data/messenger/mag/pfile",
        "ofile_path": "/data/messenger/mag/ofile"
    },
    "FIPS_CDR_SCAN": {
        "pfile_path": "/data/messenger/scan/pfile",
        "ofile_path": "/data/messenger/scan/ofile"
    }
}
```
それぞれ，MAGにはMESSENGER MAG Time-Averaged Calibrated MSO Coordinates Science Data Collectionに関連するファイルを，FIPS_CDR_SCANにはMESSENGER EPPS EPS Calibrate Scan Rate Data Collectionのファイルを配置します．    
また，ofile_pathにはダウンロードしてきたファイル，pfile_pathには変換後のバイナリーファイルのパスを記述します

今回は以下のような内容になります
```shell
{
    "MAG": {
        "pfile_path": "/Data/ASCII/MAG",
        "ofile_path": "/data/ASCII/FIPS_CDR_SCAN"
    },
    "FIPS_CDR_SCAN": {
        "pfile_path": "/Data/Binary/MAG",
        "ofile_path": "/Data/Binary/FIPS_CDR_SCAN"
    }
}
```
なお，ここでは**相対パスではなく絶対パスを記述する**ようにしてください

この設定後，MDUSでデータが利用可能になります        
なお，この操作は別のディレクトリでMDUSを利用するたびに行う必要があります

## STEP4：データの利用
上記の設定がすべて行えていれば，MDUSでデータを利用する事が可能になります

以降
* データのロード
* データの利用
* データのプロット
  
の3点を説明します

より詳細の利用方法についてはWikiを参照してください

### データのロード
MDUSでは全てのデータを`MDUS.Datas`クラスで扱うことが可能です    
`MDUS.Datas`クラスを作成した後，適切な設定を行うことでデータのロードが行えます

ここでは例として
1. MAGデータとFIPS_CDR_SCANデータを利用し，
     * MAGデータは10秒値を利用
     * 軌道番号で読み込むデータを指定
2. FIPS_CDR_SCANデータのみを利用し，
     * qualityが5以下のものを利用
     * 時刻で読み込むデータを指定
3. ダウンロードしていないデータを読み込もうとした場合

の3つを行います 

ロードしたデータの情報は`MDUS.Datas.Info()`で確認することが可能です

In [10]:
# データロードの例１
# Datasクラスの作成
sample1 = MDUS.Datas(["MAG","FIPS_CDR_SCAN"])
# MAGデータの設定
sample1.value["MAG"].LoadSetting(sec=10)
# 軌道番号の指定：199番目を指定
sample1.Load(orbit=199)
# 情報の確認
## 全体の情報
sample1.Info()
# MAGデータの情報
sample1.value["MAG"].Info()

Data Type
	Integrated Data
Input Data
	MAG
	FIPS_CDR_SCAN
Load Success
	MAG
	FIPS_CDR_SCAN
Data Type
	MAG
Second
	10
Unit
	Rm
Orbit
	199
Start Date
	2011-06-25 21:06:16
End Date
	2011-06-25 23:15:33
Original File
	/home/stream/data/messenger/mag/mso-ave/original/2011/data_file/MAGMSOSCIAVG11176_10_V08.TAB
Input File
	/home/stream/data/messenger/mag/mso-ave/2011_176_10.pkl


In [ ]:
# データロードの例2
# Datasクラスの作成
sample1 = MDUS.Datas(["FIPS_CDR_SCAN"])
# FIPS_CDR_SCANデータの設定
sample1.value["FIPS_CDR_SCAN"].LoadSetting(quality=5)
# 時刻の指定：2011-06-25 21:00:00から2011-06-25 22:00:00まで
sample1.Load(start="2011-06-25 21:00:00", end="2011-06-25 22:00:00")
# 情報の確認
## 全体の情報
sample1.Info()
# FIPS_CDR_SCANデータの情報
sample1.value["FIPS_CDR_SCAN"].Info()

Data Type
	Integrated Data
Input Data
	FIPS_CDR_SCAN
Load Success
	FIPS_CDR_SCAN
Data Type
	FIPS_CDR_SCAN
Target
	Proton
Quality
	5
Start Date
	2011-06-25 21:00:00
End Date
	2011-06-25 22:00:00
Original File
	/home/stream/data/messenger/fips/cdr/scan/original/2011/data_file/FIPS_R2011176CDR_V4.TAB
Input File
	/home/stream/data/messenger/fips/cdr/scan/2011_176_Proton.pkl


In [8]:
# データロードの例3
# 存在しないデータファイルを読み込もうとした場合
sample3 = MDUS.Datas(["MAG"])
# orbit:199の磁場20Hzデータを読み込む
sample3.value["MAG"].LoadSetting(sec='raw')
# orbit番号199のデータを読み込む
sample3.Load(orbit=199)

### データの利用
MDUSでロードしたデータは`value`属性に格納されています   
データのロードに成功した場合には`pandas.DataFrame`クラスに，失敗した場合には`None`になります    
そのため，pandasで提供されているメソッドを利用可能です

また，MDUSでもいくつかのメソッドを提供しています

ここでは
* pandasを利用した特定のデータのみの抽出
* SPICEを利用した軌道データの読み込み
* 座標変換
  
の3例を行います

In [13]:
# 例1：特定のデータのみの抽出
# sample1の磁場データから，X成分のみを抽出
# .valueでデータにアクセスし，Bx成分を取得
sample1.value["MAG"].value["Bx"]

date
2011-06-25 21:06:20    21.033
2011-06-25 21:06:30    21.033
2011-06-25 21:06:40    20.624
2011-06-25 21:06:50    29.278
2011-06-25 21:07:00    37.392
                        ...  
2011-06-25 23:14:50    26.111
2011-06-25 23:15:00    26.607
2011-06-25 23:15:10    30.745
2011-06-25 23:15:20    28.818
2011-06-25 23:15:30    26.212
Name: Bx, Length: 776, dtype: float64

In [14]:
# 例2：軌道データの取得
sample1.value["FIPS_CDR_SCAN"].GetPos()
sample1.value["FIPS_CDR_SCAN"].value

,19.9492,18.1198,16.4578,14.9482,13.5774,12.3322,11.2011,10.1738,9.2407,8.393,...,0.0683,0.062,0.0564,0.0511,0.0464,quality,mode,X_MSO,Y_MSO,Z_MSO
date,,,,,,,,,,,,,,,,,,,,,
2011-06-25 21:16:23.424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,2,1.106856,0.606392,1.026215
2011-06-25 21:16:33.424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,385800.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,2,1.096053,0.602234,1.030209
2011-06-25 21:16:42.424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,2,1.086297,0.598472,1.033772
2011-06-25 21:16:52.424,0.0,0.0,0.0,0.0,224300.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,2,1.075420,0.594273,1.037696
2011-06-25 21:17:01.424,0.0,142600.0,0.0,193900.0,0.0,0.0,0.0,338700.0,1157000.0,438000.0,...,0.0,0.0,0.0,0.0,0.0,0,2,1.065597,0.590475,1.041195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-06-25 22:44:57.424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,2,-1.328214,-0.929728,-2.898410
2011-06-25 22:45:06.424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,2,-1.326307,-0.929705,-2.904764
2011-06-25 22:45:16.424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,2,-1.324182,-0.929677,-2.911813


In [16]:
# 例3：座標系の変換
# MSO座標->MSM'座標
sample1.value["FIPS_CDR_SCAN"].CTransform('aMSM')
sample1.value["FIPS_CDR_SCAN"].value

,19.9492,18.1198,16.4578,14.9482,13.5774,12.3322,11.2011,10.1738,9.2407,8.393,...,0.0511,0.0464,quality,mode,X_MSO,Y_MSO,Z_MSO,X_aMSM,Y_aMSM,Z_aMSM
date,,,,,,,,,,,,,,,,,,,,,
2011-06-25 21:16:23.424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,2,1.106856,0.606392,1.026215,1.019430,0.744046,0.826215
2011-06-25 21:16:33.424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,385800.0,0.0,...,0.0,0.0,0,2,1.096053,0.602234,1.030209,1.009253,0.738529,0.830209
2011-06-25 21:16:42.424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,2,1.086297,0.598472,1.033772,1.000064,0.733541,0.833772
2011-06-25 21:16:52.424,0.0,0.0,0.0,0.0,224300.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,2,1.075420,0.594273,1.037696,0.989819,0.727975,0.837696
2011-06-25 21:17:01.424,0.0,142600.0,0.0,193900.0,0.0,0.0,0.0,338700.0,1157000.0,438000.0,...,0.0,0.0,0,2,1.065597,0.590475,1.041195,0.980567,0.722942,0.841195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-06-25 22:44:57.424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,2,-1.328214,-0.929728,-2.898410,-1.197250,-1.093224,-3.098410
2011-06-25 22:45:06.424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,2,-1.326307,-0.929705,-2.904764,-1.195361,-1.092956,-3.104764
2011-06-25 22:45:16.424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,2,-1.324182,-0.929677,-2.911813,-1.193258,-1.092653,-3.111813
